In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import time
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm

# Тематическое моделирование твитов.


### Преодобработка

In [3]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
def del_users(string):
    users = re.findall('@[\w]*', string)
    if users:
        result = re.sub(users[0], ' ', string)
        return result.lower()
    else:
        return string.lower()

In [5]:
data = train_df['tweet'].apply(del_users)

In [15]:
def remove_punctuation(string):
    return re.sub(r'[^\w\s]',' ', string)

In [16]:
data_6 = data_4.apply(remove_punctuation)


In [20]:
import re
import numpy as np
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
from pymystem3 import Mystem

In [21]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dany\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [55]:
data = data_6

In [60]:
data = pd.DataFrame(data)

In [61]:
stopwords_list = stopwords.words('english')

In [62]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

In [63]:
print(preprocess(data.tweet.iloc[1]))

['thanks', 'lyft', 'credit', 'cannot', 'use', 'cause', 'offer', 'wheelchair', 'vans', 'pdx', 'disapointed', 'getthanked']


In [64]:
preprocessed_text = list(tqdm(map(preprocess, data['tweet']), total=len(data)))

  0%|          | 0/31962 [00:00<?, ?it/s]

In [66]:
len(preprocessed_text)

31962

In [67]:
from gensim.models import *
from gensim import corpora

In [68]:
data['text'] = preprocessed_text
data.sample(3)

,tweet,text
3184,all of the dems that were packing the house an...,"[dems, packing, house, crossing, vote, open, p..."
5685,trumpâ s odd nye tweet to â enemiesâ trum...,"[trumpâ, odd, nye, tweet, enemiesâ, trump, odd..."
27901,happy national best friend day nationalbestfr...,"[happy, national, best, friend, day, nationalb..."


### LDA

In [69]:
dictionary = corpora.Dictionary(data['text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 

In [70]:
corpus = [dictionary.doc2bow(text) for text in data['text']]
corpora.MmCorpus.serialize('tweeter.model', corpus) 

In [79]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=7, chunksize=50, update_every=1, passes=2)

CPU times: total: 4.5 s
Wall time: 4.55 s


In [80]:

lda.show_topics(num_topics=7, num_words=10, formatted=True)

[(0,
  '0.035*"sad" + 0.027*"weekend" + 0.022*"know" + 0.022*"cute" + 0.017*"little" + 0.016*"selfie" + 0.015*"say" + 0.015*"trying" + 0.014*"made" + 0.014*"holiday"'),
 (1,
  '0.038*"new" + 0.024*"bihday" + 0.023*"fun" + 0.023*"work" + 0.022*"way" + 0.020*"first" + 0.018*"music" + 0.018*"back" + 0.015*"got" + 0.015*"healthy"'),
 (2,
  '0.043*"one" + 0.031*"summer" + 0.029*"morning" + 0.026*"bull" + 0.026*"want" + 0.024*"make" + 0.022*"friday" + 0.015*"still" + 0.015*"trump" + 0.014*"whatever"'),
 (3,
  '0.116*"love" + 0.055*"time" + 0.042*"like" + 0.030*"smile" + 0.023*"take" + 0.023*"need" + 0.021*"sunday" + 0.021*"week" + 0.019*"thank" + 0.018*"going"'),
 (4,
  '0.080*"day" + 0.055*"amp" + 0.055*"happy" + 0.032*"life" + 0.031*"get" + 0.027*"today" + 0.026*"thankful" + 0.026*"father" + 0.024*"see" + 0.024*"positive"'),
 (5,
  '0.049*"good" + 0.031*"people" + 0.021*"tomorrow" + 0.019*"follow" + 0.018*"never" + 0.017*"beautiful" + 0.017*"orlando" + 0.017*"night" + 0.014*"hea" + 0.014*"

### Визуализация

In [81]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

C:\Users\dany\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


CPU times: total: 719 ms
Wall time: 2.79 s


1 тема: повседневная жизнь

2 тема: развлечения по типу дня рождения и выходных

3 тема: взаимоотношения людей

4 тема: планы в широком понимании этого слова

5 тема: политика

6 тема: планы в широком понимании этого слова, так как кластеры достаточно сильно перечикаются с 4 темой

7 тема: твиты про плохое настроение